In [307]:
import pandas as pd
import requests
import json
import numpy as np

def searching(
    query_string,
    broad='no',
    has_ngrams='true',
    does_date_matter = 'yes',
    from_date='2000-01-01',
    to_date='2022-12-12'):

    if broad == 'no':
        if does_date_matter == 'yes':
            institution = requests.get(
                f'https://api.openalex.org/works?filter=abstract.search:{query_string},has_ngrams:{has_ngrams},from_publication_date:{from_date},to_publication_date:{to_date}&page=1&per-page=20'
            ).json()
        else:
            institution = requests.get(
                f'https://api.openalex.org/works?filter=abstract.search:{query_string},has_ngrams:{has_ngrams}&page=1&per-page=20'
            ).json()

    elif broad == 'yes':
        institution = requests.get(
            f'https://api.openalex.org/works?search={query_string}&filter=has_ngrams:{has_ngrams}&page=1&per-page=20'
        ).json()
    
    a = []
    for x in range(15):
        y = institution['results'][x]['ngrams_url']
        if x < 3:
            print(institution['results'][x]['title'])
            print(institution['results'][x]['authorships'][0]['author']['display_name'])
            print(institution['results'][x]['doi'])
            print(institution['results'][x]['relevance_score'])
            print(y,'\n')  
        z = requests.get(y).json()
        a.append(z)


    return a

_ngrams = searching('environmental flows',broad='yes')

# for idx,i in enumerate(_ngrams):
#     if idx == 0:
#         j = i['ngrams']
#         k = pd.DataFrame.from_dict(j)
#     else:
#         j = i['ngrams']
#         l = pd.DataFrame.from_dict(j)
#         k = pd.concat([k,l])

A global perspective on environmental flow assessment: emerging trends in the development and application of environmental flow methodologies for rivers
Rebecca E. Tharme
https://doi.org/10.1002/rra.736
2520.7532
https://api.openalex.org/works/W1996950224/ngrams 

Ecological responses to altered flow regimes: a literature review to inform the science and management of environmental flows
N. LeRoy Poff
https://doi.org/10.1111/j.1365-2427.2009.02272.x
2138.908
https://api.openalex.org/works/W2120716973/ngrams 

THE CHALLENGE OF PROVIDING ENVIRONMENTAL FLOW RULES TO SUSTAIN RIVER ECOSYSTEMS
Angela Arthington
https://doi.org/10.1890/1051-0761(2006)016[1311:tcopef]2.0.co;2
2039.55
https://api.openalex.org/works/W2150457759/ngrams 



In [314]:
ngram = []

for i in _ngrams:
    for j in i['ngrams']:
        ngram.append((j['ngram'].lower(),j['ngram_count']))

print(len(ngram))

b = Counter()
for ng,co in ngram:
    b[ng] += co

c = [[x,y] for x,y in b.items()]

badwords = ['add words to filter as necessary',' .$', 'et al','et$']

df = pd.DataFrame.from_records(c,columns=['ngram','ngram_count'])
df['ngram_tokens'] = df['ngram'].str.split().str.len() #https://stackoverflow.com/questions/37483470/how-to-calculate-number-of-words-in-a-string-in-dataframe
df1 = df.dropna(axis=0)
df2 = df1.loc[(df1['ngram_tokens'] > 1) & (df1['ngram_tokens'] < 5)]
df3 = df2.sort_values(by=['ngram_count','ngram_tokens'],ascending=False)

for i in badwords:
    df3 = df3[~df3['ngram'].str.contains(i)]
df3.head(n=30)

101417


,ngram,ngram_count,ngram_tokens
5020,environmental flow,463,2
5080,flow regime,345,2
11494,flow alteration,188,2
5885,ecological response,131,2
5417,flow recommendation,112,2
5501,natural flow,109,2
5058,low flow,93,2
5659,case study,65,2
5451,hydrologic alteration,64,2
6301,river flow,63,2
